# 🧪 **Liquid clustering** 🧪

En este notebook, veremos como aplicar [la característica de liquid clustering](https://docs.delta.io/latest/delta-clustering.html) a nuestras tablas para poder mejorar el rendimiento de las operaciones y consultas realizadas sobre ellas dentro de [Microsoft Fabric](https://www.vernegroup.com/actualidad/tecnologia/microsoft-fabric-liquid-clustering/).

⚠️ Ten en cuenta que para poder ejecutar la mayoría de pasos de este cuaderno, necesitarás:
- Tener un lakehouse agregado a él. Para ello, puedes seguir [los pasos indicados aquí](https://learn.microsoft.com/es-es/fabric/data-engineering/lakehouse-notebook-explore).
- Contar con el runtime 1.3 de Fabric activado en el área de trabajo.
- Haber cargado previamente el dataset de NYC_Taxi dentro de tu lakehouse. Puedes hacerlo mediante [Fabric Data Factory](https://learn.microsoft.com/es-es/fabric/data-factory/create-first-pipeline-with-sample-data).


## Crear tabla con liquid clustering configurado
---

Además del runtime de Spark 1.3, necesitamos alterar las propiedades _**minReaderVersion**_ y **_minWriterVersion_** dentro de la tabla.

🚨 Debemos hacerlo en dos instrucciones separadas o no registra bien los cambios.

In [66]:
%%sql

-- Creación de la tabla incluyendo clustering
CREATE OR REPLACE TABLE nyc_taxi_clustered (
    vendorID int, 
    lpepPickupDatetime timestamp,
    lpepDropoffDatetime timestamp,
    passengerCount int,
    tripDistance double,
    puLocationId string,
    doLocationId string,
    pickupLongitude double,
    pickupLatitude double,
    dropoffLongitude double,
    dropoffLatitude double,
    rateCodeId int,
    storeAndFwdFlag string,
    paymentType int,
    fareAmount double,
    extra double,
    mtaTax double,
    improvementSurcharge string,
    tipAmount double,
    tollsAmount double,
    ehailFee double,
    totalAmount double,
    tripType int
    ) 
CLUSTER BY (
    vendorID,
    lpepPickupDatetime,
    lpepDropoffDatetime
);

-- Establecer las propiedades necesarias
ALTER TABLE nyc_taxi_clustered SET TBLPROPERTIES (
    'delta.minReaderVersion' = '2',
    'delta.minWriterVersion' = '7'
);

StatementMeta(, 7da31ac3-36e8-4215-ac8d-c60f2ba6ed8d, 78, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

### Comprobar propiedades y clustering configurado
---

Gracias al comando **_DESCRIBE DETAIL_**, podremos comprobar la configuración establecida dentro de los campos **_clusteringColumns_**, **_minReaderVersion_**, **_minWriterVersion_** y **_tableFeatures_**.

In [73]:
%%sql

DESCRIBE DETAIL nyc_taxi_clustered;

StatementMeta(, 7da31ac3-36e8-4215-ac8d-c60f2ba6ed8d, 85, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 15 fields>

### Cargamos el contenido del dataset a la tabla
---

Para cargar el contenido, disponemos de varias formas:
- Podemos realizar un volcado mediante el modo **_append_**
- Podemos realizar un **_SELECT INTO_** a través de Spark SQL
- El comando merge también respeta la configuración establecida

🚨 Si volcamos datos en modo overwrite, la tabla se recrea y pierde la configuración establecida de liquid clustering.

In [68]:
# Leemos el contenido del dataset
nyc_taxi_df = spark.read.parquet("Files/raw/20240930/NYC_taxi.parquet")

# Inserción en la tabla vía append
#nyc_taxi_df.write.mode("append").saveAsTable("nyc_taxi_clustered")

# Inserción vía Spark SQL
spark.sql("INSERT INTO nyc_taxi_clustered SELECT * FROM {spark_df}",\
    spark_df = nyc_taxi_df)

StatementMeta(, 7da31ac3-36e8-4215-ac8d-c60f2ba6ed8d, 80, Finished, Available, Finished)

DataFrame[]

### Comprobamos que todo sigue como debe
---

In [69]:
%%sql

DESCRIBE DETAIL nyc_taxi_clustered;

StatementMeta(, 7da31ac3-36e8-4215-ac8d-c60f2ba6ed8d, 81, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 15 fields>